In [1]:
import os
import numpy as np
import torch
from models_brain import C8SteerableCNN
from neuralpredictors.layers.readouts import SpatialXFeatureLinear
from train_utils import *
from model_utils import *
from dataloaders import Dataset_visual
from itertools import repeat
from utils import save_checkpoint

In [2]:
# Generators
roi_type = 'face'
roi = 4
training_set = Dataset_visual(mode = 'train', roi_type = roi_type, roi = roi)
training_generator = torch.utils.data.DataLoader(training_set,  **params)

validation_set = Dataset_visual(mode = 'val', roi_type = roi_type, roi = roi)
validation_generator = torch.utils.data.DataLoader(validation_set,  **params)


n_neurons = training_set.responses.shape[1]
print('Number of neurons', n_neurons)

Number of neurons 3053


In [3]:
n_feats = 48
core = C8SteerableCNN(n_feats = n_feats)
readout = SpatialXFeatureLinear(core(torch.randn(1, 3, 224, 224)).size()[1:], n_neurons,  bias = True)  
model = Encoder(core, readout)
model.cuda()   

/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:61: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at  /pytorch/aten/src/ATen/native/IndexingUtils.h:25.)
  sampled_basis = sampled_basis[mask, ...]


Encoder(
  (core): C8SteerableCNN(
    (block1): SequentialModule(
      (0): R2Conv([8-Rotations: {irrep_0, irrep_0, irrep_0}], [8-Rotations: {regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular}], kernel_size=5, stride=1, padding=1, bias=False)
      (1): InnerBatchNorm([8-Rotations: {regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, regular, 

In [4]:
model_dir = './saved_models'
if not os.path.isdir(model_dir):
    os.mkdir(model_dir)
model_base = 'ho_%d_%s_%d_%s_%s' % (n_feats, roi_type, roi, 'scratch', 'factorized')

    
schedule = [1e-4]
criterion = masked_MSEloss 
best_corr = 0
patience = 20
iter_tracker = 0 
accumulate_gradient= 4
n_epochs = 100

############## Start training ##########
for opt, lr in zip(repeat(torch.optim.Adam), schedule):
    print('Training with learning rate', lr)
    optimizer = opt(model.parameters(), lr=lr)
    optimizer.zero_grad()
    iteration = 0
    restore_file = 'best_' + model_base
    restore_path = os.path.join(model_dir, restore_file + '.pth.tar')



    assert accumulate_gradient > 0, 'accumulate_gradient needs to be > 0'
    for epoch in range(n_epochs):
        for x_batch, y_batch in training_generator:

            obj = full_objective(model, x_batch.cuda().float(), y_batch.cuda().float(), criterion)
            obj.backward()

            iteration += 1 

            if iteration % accumulate_gradient == accumulate_gradient - 1:
                optimizer.step()
                optimizer.zero_grad()
            if iteration % 100 == 0:
                model.eval()
                true, preds = compute_predictions(validation_generator, model)

                val_corr = compute_scores(true, preds)
                print('Val correlation:', val_corr)
                model.train()
                is_best = val_corr >= best_corr
                if is_best:
                    best_corr = val_corr.copy()
                    iter_tracker = 0        
                    save_checkpoint({'epoch': epoch + 1,
                                           'state_dict': model.state_dict(),
                                           'optim_dict' : optimizer.state_dict()},
                                           is_best = is_best,
                                           checkpoint = model_dir, model_str = model_base)
                else:
                    iter_tracker += 1
                    if iter_tracker == patience: 
                        print('Training complete')
                        break 
        if iter_tracker == patience: 
                        print('Training complete')
                        break  



Training with learning rate 0.0001


/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return default_collate([torch.as_tensor(b) for b in batch])
/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable befor

/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return default_collate([torch.as_tensor(b) for b in batch])
/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable befor

/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return default_collate([torch.as_tensor(b) for b in batch])
/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable befor

Val correlation: 0.0963899862200478
Checkpoint Directory exists! 


...

Val correlation: 0.05786704327601946


/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return default_collate([torch.as_tensor(b) for b in batch])
/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable befor

/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return default_collate([torch.as_tensor(b) for b in batch])
/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable befor

/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return default_collate([torch.as_tensor(b) for b in batch])
/home/mk2299/.conda/envs/dissect/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py:63: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable befor

Val correlation: 0.05781252049322666
